In [1]:
%pip install ../../../

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=67515 sha256=bc19ba9162d2fe9bef6ee08b9c15b02df5613daf9a0501e510045238dc554b31
  Stored in directory: /tmp/pip-ephem-wheel-cache-7b__dsfq/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="train",
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

In [4]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T


valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()

In [5]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [6]:
from spuco.models import model_factory 
model = model_factory("mlp", trainset[0][0].shape, trainset.num_spurious).to(device)
from torch.optim import SGD
from spuco.invariant_train import ERM

erm = ERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
erm.train()

Epoch 0: 100%|██████████| 751/751 [00:02<00:00, 311.70batch/s, accuracy=100.0%, loss=0.00805] 


In [7]:
from collections import Counter 

Counter(valset.spurious).most_common()

[(0, 2402), (1, 2401), (3, 2398), (2, 2398), (4, 2397)]

In [8]:
from spuco.group_inference import SSA
from spuco.utils import SpuriousTargetDataset

ssa = SSA(
    spurious_unlabeled_dataset=trainset,
    spurious_labeled_dataset=SpuriousTargetDataset(valset, valset.spurious),
    model=model_factory("lenet", trainset[0][0].shape, trainset.num_spurious).to(device),
    labeled_valset_size=0.1,
    num_iters=1000,
    tau_g_min=0.3,
    device=device,
    verbose=True
)

In [9]:
group_partition = ssa.infer_groups()
for key in sorted(group_partition.keys()):
    print(key, len(group_partition[key]))

Training SSA Model 2: 100%|██████████| 1000/1000 [00:16<00:00, 59.11it/s, loss=nan, val_acc=19.7]


(0, 0) 10024
(0, 1) 27
(0, 2) 26
(0, 3) 28
(0, 4) 28
(1, 0) 36
(1, 1) 9561
(1, 2) 16
(1, 3) 35
(1, 4) 24
(2, 0) 28
(2, 1) 18
(2, 2) 7973
(2, 3) 959
(2, 4) 33
(3, 0) 15
(3, 1) 25
(3, 2) 28
(3, 3) 9652
(3, 4) 27
(4, 0) 26
(4, 1) 24
(4, 2) 25
(4, 3) 20
(4, 4) 9346


In [10]:
from spuco.evaluate import Evaluator 

evaluator = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 100.0
Group (0, 1) Test Accuracy: 11.11111111111111
Group (0, 2) Test Accuracy: 0.0
Group (0, 3) Test Accuracy: 0.0
Group (0, 4) Test Accuracy: 14.285714285714286
Group (1, 0) Test Accuracy: 38.888888888888886
Group (1, 1) Test Accuracy: 100.0
Group (1, 2) Test Accuracy: 0.0
Group (1, 3) Test Accuracy: 0.0
Group (1, 4) Test Accuracy: 0.0
Group (2, 0) Test Accuracy: 0.0
Group (2, 1) Test Accuracy: 0.0
Group (2, 2) Test Accuracy: 100.0
Group (2, 3) Test Accuracy: 98.22732012513035
Group (2, 4) Test Accuracy: 15.151515151515152
Group (3, 0) Test Accuracy: 0.0
Group (3, 1) Test Accuracy: 0.0
Group (3, 2) Test Accuracy: 0.0
Group (3, 3) Test Accuracy: 99.96891835888935
Group (3, 4) Test Accuracy: 0.0
Group (4, 0) Test Accuracy: 0.0
Group (4, 1) Test Accuracy: 0.0
Group (4, 2) Test Accuracy: 0.0
Group (4, 3) Test Accuracy: 0.0
Group (4, 4) Test Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 11.11111111111111,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 14.285714285714286,
 (1, 0): 38.888888888888886,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 100.0,
 (2, 3): 98.22732012513035,
 (2, 4): 15.151515151515152,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 99.96891835888935,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

In [11]:
from torch.optim import SGD
from spuco.invariant_train import GroupBalanceERM, ClassBalanceERM
from spuco.models import model_factory 

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
group_balance_erm = GroupBalanceERM(
    model=model,
    num_epochs=5,
    trainset=trainset,
    group_partition=group_partition,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
group_balance_erm.train()

Epoch 4: 100%|██████████| 751/751 [00:01<00:00, 417.34batch/s, accuracy=100.0%, loss=0.0225]  


In [12]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 99.05437352245863
Group (0, 1) Test Accuracy: 85.34278959810875
Group (0, 2) Test Accuracy: 93.3806146572104
Group (0, 3) Test Accuracy: 91.96217494089835
Group (0, 4) Test Accuracy: 89.36170212765957
Group (1, 0) Test Accuracy: 90.70904645476773
Group (1, 1) Test Accuracy: 92.66503667481662
Group (1, 2) Test Accuracy: 83.57843137254902
Group (1, 3) Test Accuracy: 88.48039215686275
Group (1, 4) Test Accuracy: 89.70588235294117
Group (2, 0) Test Accuracy: 81.6
Group (2, 1) Test Accuracy: 74.13333333333334
Group (2, 2) Test Accuracy: 94.66666666666667
Group (2, 3) Test Accuracy: 72.0
Group (2, 4) Test Accuracy: 78.6096256684492
Group (3, 0) Test Accuracy: 87.93969849246231
Group (3, 1) Test Accuracy: 87.6574307304786
Group (3, 2) Test Accuracy: 81.61209068010075
Group (3, 3) Test Accuracy: 86.90176322418137
Group (3, 4) Test Accuracy: 84.38287153652394
Group (4, 0) Test Accuracy: 88.16120906801008
Group (4, 1) Test Accuracy: 87.40554156171285
Group (4, 2) Test

{(0, 0): 99.05437352245863,
 (0, 1): 85.34278959810875,
 (0, 2): 93.3806146572104,
 (0, 3): 91.96217494089835,
 (0, 4): 89.36170212765957,
 (1, 0): 90.70904645476773,
 (1, 1): 92.66503667481662,
 (1, 2): 83.57843137254902,
 (1, 3): 88.48039215686275,
 (1, 4): 89.70588235294117,
 (2, 0): 81.6,
 (2, 1): 74.13333333333334,
 (2, 2): 94.66666666666667,
 (2, 3): 72.0,
 (2, 4): 78.6096256684492,
 (3, 0): 87.93969849246231,
 (3, 1): 87.6574307304786,
 (3, 2): 81.61209068010075,
 (3, 3): 86.90176322418137,
 (3, 4): 84.38287153652394,
 (4, 0): 88.16120906801008,
 (4, 1): 87.40554156171285,
 (4, 2): 77.8337531486146,
 (4, 3): 89.8989898989899,
 (4, 4): 90.9090909090909}

In [13]:
evaluator.worst_group_accuracy

((2, 3), 72.0)

In [14]:
evaluator.average_accuracy

92.79891904617348

In [15]:
evaluator.evaluate_spurious_task()

22.58